In [1]:
import os
import numpy as np
import pandas as pd
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
import time
import matplotlib.pyplot as plt
import cudf 

In [2]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

# Filter valid

In [3]:
merged_candidates_feature = pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_no_hist_feature.parquet', engine='pyarrow')

In [4]:
valid_sessions = pd.read_csv('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv')

In [5]:
valid_sessions.iloc[12]

prev_items    ['B00ET1WZUO' 'B00ET1X0TO']
next_item                      B079RLJDK8
locale                                 UK
Name: 12, dtype: object

In [7]:
merged_candidates_feature

,sess_id,sess_locale,product,target,sasrec_scores_2,sasrec_normalized_scores_2,product_freq,gru4rec_scores,gru4rec_normalized_scores,sess_avg_price,...,all_items_co_graph_count_0,seqmlp_scores,seqmlp_normalized_scores,desc_BM25_scores,normalized_all_items_co_graph_count_1,all_items_co_graph_count_1,normalized_all_items_co_graph_count_2,all_items_co_graph_count_2,narm_scores,normalized_narm_scores
0,0,UK,B000OPPVCS,0.0,11.972421,2.286162e-04,104,6.484859,3.816029e-05,7.388571,...,2,8.378721,1.007163e-05,362.381012,0.003781,1.000000,0.006452,1,16.883379,2.761682e-04
1,0,UK,B000V599Y2,0.0,13.152878,7.443427e-04,37,4.342063,4.477209e-06,7.388571,...,3,7.534612,4.330198e-06,64.592583,0.000000,0.000000,0.000000,0,13.755573,1.209999e-05
2,0,UK,B0018HH444,0.0,5.606023,3.928400e-07,7,3.220763,1.458925e-06,7.388571,...,1,11.199949,1.691779e-04,67.996689,0.003781,1.000000,0.006452,1,9.759952,2.225914e-07
3,0,UK,B0079JI4DU,0.0,0.000000,1.443945e-09,67,0.000000,5.824698e-08,7.388571,...,2,0.000000,2.313489e-09,0.000000,0.001891,0.500000,0.000000,0,0.000000,1.284740e-11
4,0,UK,B0079JI4EY,0.0,0.000000,1.443945e-09,77,0.000000,5.824698e-08,7.388571,...,2,0.000000,2.313489e-09,0.000000,0.003781,1.000000,0.006452,1,0.000000,1.284740e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84407334,361580,DE,B0BB7XV97M,0.0,9.117821,6.077226e-05,56,9.268379,1.396883e-05,32.424000,...,2,13.965775,2.930834e-03,0.000000,0.000000,0.000000,0.000000,0,9.397533,2.219567e-04
84407335,361580,DE,B0BB7YSRBX,0.0,9.163816,6.363281e-05,58,7.047796,1.516259e-06,32.424000,...,1,10.212118,6.867505e-05,0.000000,0.000000,0.000000,0.000000,0,8.080528,5.947030e-05
84407336,361580,DE,B0BB7ZMGY8,0.0,11.256460,5.158278e-04,452,9.359167,1.529639e-05,32.424000,...,23,12.275789,5.408041e-04,0.000000,0.010152,1.226190,0.000000,0,10.893625,9.908607e-04
84407337,361580,DE,B0BD4CP7N3,0.0,-3.778687,1.523433e-10,1,-0.593306,7.282568e-10,32.424000,...,0,-2.456360,2.162463e-10,36.028561,0.000000,0.000000,0.000000,0,2.628504,2.549859e-07


In [8]:
drop_index = merged_candidates_feature.query('all_items_co_graph_count_0==0 & product_freq>200').index
merged_candidates_feature_droped = merged_candidates_feature.drop(drop_index, axis=0)
merged_candidates_feature_droped.reset_index(drop=True, inplace=True)

In [9]:
# cast_dtype(merged_candidates_feature_droped)
merged_candidates_feature_droped.to_parquet('./candidates/merged_candidates_no_hist_pad_feature.parquet')

In [10]:
merged_candidates_feature_droped

,sess_id,sess_locale,product,target,sasrec_scores_2,sasrec_normalized_scores_2,product_freq,gru4rec_scores,gru4rec_normalized_scores,sess_avg_price,...,all_items_co_graph_count_0,seqmlp_scores,seqmlp_normalized_scores,desc_BM25_scores,normalized_all_items_co_graph_count_1,all_items_co_graph_count_1,normalized_all_items_co_graph_count_2,all_items_co_graph_count_2,narm_scores,normalized_narm_scores
0,0,UK,B000OPPVCS,0.0,11.972421,2.286162e-04,104,6.484859,3.816029e-05,7.388571,...,2,8.378721,1.007163e-05,362.381012,0.003781,1.000000,0.006452,1,16.883379,2.761682e-04
1,0,UK,B000V599Y2,0.0,13.152878,7.443427e-04,37,4.342063,4.477209e-06,7.388571,...,3,7.534612,4.330198e-06,64.592583,0.000000,0.000000,0.000000,0,13.755573,1.209999e-05
2,0,UK,B0018HH444,0.0,5.606023,3.928400e-07,7,3.220763,1.458925e-06,7.388571,...,1,11.199949,1.691779e-04,67.996689,0.003781,1.000000,0.006452,1,9.759952,2.225914e-07
3,0,UK,B0079JI4DU,0.0,0.000000,1.443945e-09,67,0.000000,5.824698e-08,7.388571,...,2,0.000000,2.313489e-09,0.000000,0.001891,0.500000,0.000000,0,0.000000,1.284740e-11
4,0,UK,B0079JI4EY,0.0,0.000000,1.443945e-09,77,0.000000,5.824698e-08,7.388571,...,2,0.000000,2.313489e-09,0.000000,0.003781,1.000000,0.006452,1,0.000000,1.284740e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69855610,361580,DE,B0BB7XV97M,0.0,9.117821,6.077226e-05,56,9.268379,1.396883e-05,32.424000,...,2,13.965775,2.930834e-03,0.000000,0.000000,0.000000,0.000000,0,9.397533,2.219567e-04
69855611,361580,DE,B0BB7YSRBX,0.0,9.163816,6.363281e-05,58,7.047796,1.516259e-06,32.424000,...,1,10.212118,6.867505e-05,0.000000,0.000000,0.000000,0.000000,0,8.080528,5.947030e-05
69855612,361580,DE,B0BB7ZMGY8,0.0,11.256460,5.158278e-04,452,9.359167,1.529639e-05,32.424000,...,23,12.275789,5.408041e-04,0.000000,0.010152,1.226190,0.000000,0,10.893625,9.908607e-04
69855613,361580,DE,B0BD4CP7N3,0.0,-3.778687,1.523433e-10,1,-0.593306,7.282568e-10,32.424000,...,0,-2.456360,2.162463e-10,36.028561,0.000000,0.000000,0.000000,0,2.628504,2.549859e-07


# Filter test

In [11]:
merged_candidates_test_feature = pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_test_no_hist_feature.parquet', engine='pyarrow')

In [12]:
drop_index = merged_candidates_test_feature.query('all_items_co_graph_count_0==0 & product_freq>200').index
merged_candidates_test_feature_droped = merged_candidates_test_feature.drop(drop_index, axis=0)
merged_candidates_test_feature_droped.reset_index(drop=True, inplace=True)

In [13]:
# cast_dtype(merged_candidates_test_feature_droped)
merged_candidates_test_feature_droped.to_parquet('./candidates/merged_candidates_test_no_hist_pad_feature.parquet')

In [14]:
merged_candidates_test_feature

,sess_id,sess_locale,product,sasrec_scores_2,sasrec_normalized_scores_2,gru4rec_scores,gru4rec_normalized_scores,product_freq,sess_avg_price,product_price,...,all_items_co_graph_count_0,seqmlp_scores,seqmlp_normalized_scores,desc_BM25_scores,normalized_all_items_co_graph_count_1,all_items_co_graph_count_1,normalized_all_items_co_graph_count_2,all_items_co_graph_count_2,narm_scores,normalized_narm_scores
0,0,DE,4088833651,0.000000,2.975813e-09,0.000000,1.580065e-09,828,25.195269,36.761604,...,0,0.000000,2.554478e-10,0.000000,0.000000,0.000000,0.000000,0,0.000000,5.463282e-09
1,0,DE,B000H6W2GW,0.000000,2.975813e-09,0.000000,1.580065e-09,875,25.195269,36.761604,...,0,0.000000,2.554478e-10,0.000000,0.000000,0.000000,0.000000,0,0.000000,5.463282e-09
2,0,DE,B000JG2RAG,7.665308,6.347557e-06,8.104032,5.226502e-06,24,25.195269,23.190001,...,0,8.786958,1.672744e-06,67.792648,0.000000,0.000000,0.000000,0,9.265436,5.772732e-05
3,0,DE,B000RYSOUW,-2.951060,1.555882e-10,-2.857798,9.068785e-11,5,25.195269,6.900000,...,0,-3.325048,9.188664e-12,170.360580,0.000000,0.000000,0.000000,0,-3.215176,2.193414e-10
4,0,DE,B000UGZVQM,3.977920,1.589257e-07,4.688567,1.717488e-07,4,25.195269,21.990000,...,0,5.540127,6.506522e-08,71.169296,0.000000,0.000000,0.000000,0,5.534946,1.384364e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69428426,316970,UK,B0BJCTH4NH,11.327528,1.041200e-04,10.629994,3.818184e-04,74,16.950001,5.800000,...,16,11.838901,9.762144e-04,164.803131,0.002540,1.285714,0.003086,1,8.999118,7.229968e-05
69428427,316970,UK,B0BJTQQWLG,5.604142,3.403292e-07,6.052083,3.923694e-06,6,16.950001,9.880000,...,1,4.890683,9.375031e-07,303.665985,0.000494,0.250000,0.000000,0,5.973598,3.508891e-06
69428428,316970,UK,B0BJV3RL4H,9.146974,1.176336e-05,7.667603,1.973815e-05,7,16.950001,22.097065,...,1,10.187823,1.872800e-04,226.131516,0.001976,1.000000,0.003086,1,7.151886,1.139970e-05
69428429,316970,UK,B0BK7SPC84,-10.383047,3.879279e-14,-6.356799,1.601719e-11,0,16.950001,5.960000,...,0,-4.160688,1.099036e-10,312.603607,0.000000,0.000000,0.000000,0,-2.646061,6.334315e-10


In [15]:
merged_candidates_test_feature_droped

,sess_id,sess_locale,product,sasrec_scores_2,sasrec_normalized_scores_2,gru4rec_scores,gru4rec_normalized_scores,product_freq,sess_avg_price,product_price,...,all_items_co_graph_count_0,seqmlp_scores,seqmlp_normalized_scores,desc_BM25_scores,normalized_all_items_co_graph_count_1,all_items_co_graph_count_1,normalized_all_items_co_graph_count_2,all_items_co_graph_count_2,narm_scores,normalized_narm_scores
0,0,DE,B000JG2RAG,7.665308,6.347557e-06,8.104032,5.226502e-06,24,25.195269,23.190001,...,0,8.786958,1.672744e-06,67.792648,0.000000,0.000000,0.000000,0,9.265436,5.772732e-05
1,0,DE,B000RYSOUW,-2.951060,1.555882e-10,-2.857798,9.068785e-11,5,25.195269,6.900000,...,0,-3.325048,9.188664e-12,170.360580,0.000000,0.000000,0.000000,0,-3.215176,2.193414e-10
2,0,DE,B000UGZVQM,3.977920,1.589257e-07,4.688567,1.717488e-07,4,25.195269,21.990000,...,0,5.540127,6.506522e-08,71.169296,0.000000,0.000000,0.000000,0,5.534946,1.384364e-06
3,0,DE,B000Z6JN7K,6.600063,2.187643e-06,7.367919,2.503351e-06,7,25.195269,13.170000,...,0,9.692825,4.138498e-06,128.350281,0.000000,0.000000,0.000000,0,8.766117,3.503724e-05
4,0,DE,B003CYK6FU,3.741957,1.255211e-07,6.754154,1.355086e-06,4,25.195269,11.990000,...,0,1.844546,1.615766e-09,142.318924,0.000000,0.000000,0.000000,0,5.671884,1.587529e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61646301,316970,UK,B0BJCTH4NH,11.327528,1.041200e-04,10.629994,3.818184e-04,74,16.950001,5.800000,...,16,11.838901,9.762144e-04,164.803131,0.002540,1.285714,0.003086,1,8.999118,7.229968e-05
61646302,316970,UK,B0BJTQQWLG,5.604142,3.403292e-07,6.052083,3.923694e-06,6,16.950001,9.880000,...,1,4.890683,9.375031e-07,303.665985,0.000494,0.250000,0.000000,0,5.973598,3.508891e-06
61646303,316970,UK,B0BJV3RL4H,9.146974,1.176336e-05,7.667603,1.973815e-05,7,16.950001,22.097065,...,1,10.187823,1.872800e-04,226.131516,0.001976,1.000000,0.003086,1,7.151886,1.139970e-05
61646304,316970,UK,B0BK7SPC84,-10.383047,3.879279e-14,-6.356799,1.601719e-11,0,16.950001,5.960000,...,0,-4.160688,1.099036e-10,312.603607,0.000000,0.000000,0.000000,0,-2.646061,6.334315e-10
